- Source code from blog: https://quangtiencs.com/posts/md_notebook/duckdb/
- Change printing format for easy-to-read in Jupyter lab.

# 1. Multiple sources and easy to integrate

In [1]:
import duckdb
con = duckdb.connect()
# con = duckdb.connect("duckdb.db")

In [2]:
query_result = con.query("""
CREATE OR REPLACE TABLE titanic AS
SELECT * FROM 'titanic.csv';

SELECT * 
FROM titanic
LIMIT 7
""")
query_result

┌─────────────┬──────────┬────────┬───────────┬───┬───────┬──────────────────┬─────────┬─────────┬──────────┐
│ PassengerId │ Survived │ Pclass │   Lname   │ … │ Parch │      Ticket      │  Fare   │  Cabin  │ Embarked │
│    int64    │  int64   │ int64  │  varchar  │   │ int64 │     varchar      │ double  │ varchar │ varchar  │
├─────────────┼──────────┼────────┼───────────┼───┼───────┼──────────────────┼─────────┼─────────┼──────────┤
│           1 │        0 │      3 │ Braund    │ … │     0 │ A/5 21171        │    7.25 │ NULL    │ S        │
│           2 │        1 │      1 │ Cumings   │ … │     0 │ PC 17599         │ 71.2833 │ C85     │ C        │
│           3 │        1 │      3 │ Heikkinen │ … │     0 │ STON/O2. 3101282 │   7.925 │ NULL    │ S        │
│           4 │        1 │      1 │ Futrelle  │ … │     0 │ 113803           │    53.1 │ C123    │ S        │
│           5 │        0 │      3 │ Allen     │ … │     0 │ 373450           │    8.05 │ NULL    │ S        │
│         

In [3]:
df = query_result.df()
print(f"Type: {type(df)}")

Type: <class 'pandas.core.frame.DataFrame'>


In [4]:
print(f"Shape: { df.shape }")

Shape: (7, 13)


# 2. Simplified Pivot Statement

In [5]:
pivot_result = con.query("""
PIVOT titanic ON Sex USING COUNT(*) GROUP BY  Survived
""")

print(pivot_result.df().to_markdown(tablefmt='psql'))

+----+------------+----------+--------+
|    |   Survived |   female |   male |
|----+------------+----------+--------|
|  0 |          0 |       16 |     86 |
|  1 |          1 |       40 |     14 |
+----+------------+----------+--------+


# 3. Join syntax is less messy compared with Pandas dataframe

In [6]:
import pandas as pd

left_df = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)
left_df

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [7]:
right_df = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)
right_df

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


In [8]:
pandas_join_result = pd.merge(left_df, right_df, on="key")
pandas_join_result

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


In [9]:
result = con.query("""
SELECT *
FROM left_df JOIN right_df ON left_df.key=right_df.key
""")

result

┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│   key   │    A    │    B    │   key   │    C    │    D    │
│ varchar │ varchar │ varchar │ varchar │ varchar │ varchar │
├─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ K0      │ A0      │ B0      │ K0      │ C0      │ D0      │
│ K1      │ A1      │ B1      │ K1      │ C1      │ D1      │
│ K2      │ A2      │ B2      │ K2      │ C2      │ D2      │
│ K3      │ A3      │ B3      │ K3      │ C3      │ D3      │
└─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

# 4. Geospatial analytics

In [10]:
con.query("""
INSTALL spatial;
LOAD spatial;

CREATE TABLE singapore_geometry AS 
SELECT * 
FROM ST_Read('singapore.json');
""")

geometry_df = con.query("""
SELECT COUNTRY, NAME_1, geom
FROM singapore_geometry
""")

geometry_df

┌───────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────┐
│  COUNTRY  │   NAME_1   │                                            geom                                             │
│  varchar  │  varchar   │                                          geometry                                           │
├───────────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Singapore │ Central    │ MULTIPOLYGON (((103.8478 1.2222, 103.8489 1.2231, 103.8517 1.2203, 103.8504 1.2141, 103.8…  │
│ Singapore │ East       │ MULTIPOLYGON (((104.0242 1.3656, 104.0386 1.3569, 104.0364 1.3533, 104.0333 1.3536, 104.0…  │
│ Singapore │ North      │ MULTIPOLYGON (((103.8282 1.3665, 103.8271 1.3627, 103.8251 1.3635, 103.8243 1.3618, 103.8…  │
│ Singapore │ North-East │ MULTIPOLYGON (((103.8959 1.3374, 103.8889 1.3336, 103.8851 1.3409, 103.8773 1.3443, 103.8…  │
│ Singapore │ West       │ MULTI

In [11]:
which_region_df = con.query("""
SELECT COUNTRY, NAME_1
FROM singapore_geometry
WHERE ST_Contains(geom, ST_Point(103.7764, 1.2966))
""")
print(which_region_df.df().to_markdown(tablefmt='psql'))

+----+-----------+----------+
|    | COUNTRY   | NAME_1   |
|----+-----------+----------|
|  0 | Singapore | Central  |
+----+-----------+----------+


# 5. Benchmark

There are some benchmarks of Polars and DuckDB:

- **DuckDB (database-like ops benchmark):** https://duckdblabs.github.io/db-benchmark/
- **Polars (TPCH Benchmark):** https://pola.rs/posts/benchmarks/